# Q-Learning : Solution for CartPole

In [1]:
#!pip install gym

## Insert Needed Library

gym -> CartPole 문제를 위한 환경을 구성!  

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

**기본적인 gym Environment 명령어**

gym  

- .make()  
원하는 환경을 만든다.  

- .reset()  
초기 State 값을 반환한다.  

- .render()  
그래픽 환경을 시작한다.  

- .close()  
그래픽 환경을 종료한다.  
- .action_space
    - .smaple() : action을 랜덤으로 가져옴 
    - .n : action의 개수를 반환함  
    
- .step()  
action을 인자로 주고 다음 스테이트에 관한 정보를 반환한다.  

- .observation_space()
    - .shape : State 의 Shape을 반환
    - .high : State를 구성하는 각 property의 최대값 출력
    - .low : State를 구성하는 각 property의 최소값 출력

In [3]:
import gym

env = gym.make('CartPole-v0')

In [4]:
observation = env.reset() #observation return ..
print(observation)

[-0.0223288  -0.00283242  0.00467966 -0.00785557]


In [5]:
for _ in range(2): # episode 2번 생성

    env.reset()

    for _ in range(50):
        env.render()
        action = env.action_space.sample() # action 랜덤으로 가져옴
        observation, reward, done, info = env.step(action) # Transition 일으킴

        print(observation, reward, done, info)

        if done:
            break

[ 0.01570962  0.19810006 -0.02273824 -0.3018503 ] 1.0 False {}
[ 0.01967162  0.39353858 -0.02877525 -0.60161677] 1.0 False {}
[ 0.02754239  0.19883072 -0.04080758 -0.31813463] 1.0 False {}
[ 0.03151901  0.3945094  -0.04717028 -0.6234024 ] 1.0 False {}
[ 0.03940919  0.20007669 -0.05963833 -0.34594066] 1.0 False {}
[ 0.04341073  0.39599403 -0.06655714 -0.65681748] 1.0 False {}
[ 0.05133061  0.59197627 -0.07969349 -0.96969331] 1.0 False {}
[ 0.06317013  0.78807234 -0.09908735 -1.28630894] 1.0 False {}
[ 0.07893158  0.98430586 -0.12481353 -1.60829943] 1.0 False {}
[ 0.0986177   1.18066247 -0.15697952 -1.9371431 ] 1.0 False {}
[ 0.12223095  1.37707534 -0.19572238 -2.27410242] 1.0 False {}
[ 0.14977245  1.57340856 -0.24120443 -2.62015521] 1.0 True {}
[-0.01842126 -0.18979839  0.02538217  0.34734938] 1.0 False {}
[-0.02221723  0.00495351  0.03232916  0.06277719] 1.0 False {}
[-0.02211816  0.19959738  0.0335847  -0.21953294] 1.0 False {}
[-0.01812621  0.00401186  0.02919404  0.08355196] 1.0 Fa

In [6]:
env.action_space.sample()
env.step?

In [7]:
print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

Box(4,)
Discrete(2)
0


In [8]:
print(env.observation_space.low)
print(env.observation_space.high)

[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [9]:
n_observation = env.observation_space.shape
n_actions = env.action_space.n

print(n_observation)
print(n_actions)

(4,)
2


## Init the Variables for CartPole

In [10]:
# State의 각 요소들을 각 10부분으로 나눈다.
n_bins_pos = 10
n_bins_vel = 10
n_bins_ang = 10
n_bins_anv = 10

In [11]:
# 모든 경우의 수를 적용하여 State의 개수 / Action의 개수를 선언
n_states = n_bins_pos*n_bins_vel*n_bins_ang*n_bins_anv
n_action = 2

# Q-Learning 을 위한 Q-table 생성 : 해당 테이블을 Q-Learning기법을 사용하여 업데이트 목적
Q_table = np.random.uniform(0, 1,(n_states, n_actions)) # 0~1 값 & shape

In [12]:
print(Q_table, Q_table.shape)

[[0.93386109 0.06616627]
 [0.50003217 0.10298524]
 [0.77737097 0.47632901]
 ...
 [0.79319662 0.43206754]
 [0.60099723 0.94285235]
 [0.04515138 0.41372704]] (10000, 2)


In [13]:
env.reset()
observation, _, _, _ = env.step(0)
env.close()

pos, vel, ang, anv = observation
print(observation)

[ 0.02752572 -0.15437297  0.00359311  0.33313387]


## Mapping Discrete State!

**State의 각 요소들을 10부분으로 나눔으로, 어떤 State가 현재 State인지 Mapping하기 위한 함수 이다.**  


- np.where() -> 괄호 안에 조건문과 일치하는 요소의 인덱스를 반환
- np.histogram() -> array와 bins(구간) 을 받는 다. range는 범위를 말하는 데 보통 (min ~ max) 이다.  
bins 가 정수 일때, 정수크기 만큼의 구간으로 (정해진 범위내에서) 나눈다.  
해당 범위 안에 속한 값들의 개수를 반환한다.    
- np.clip() -> number , min, max(범위) 값을 차례로 받고, 해당 number가 정해진 범위를 벗어나지 않도록 한다. 벗어날 경우 최소 혹은 최대값으로 변경한다.  

In [14]:
import numpy as np
array = np.random.uniform(0, 1, (4))
print(array)

[0.98337134 0.17087763 0.23394778 0.5011882 ]


In [15]:

hist, bins = np.histogram(array[0], bins=10, range=(-2, 2))
print(hist)
print(bins)

states = np.zeros([10, 10, 10, 10])
states[1, 2, 3, 4] = 1

print(np.where(states == 1))

states = states.reshape(-1, 1)
print(np.where(states == 1))

[0 0 0 0 0 0 0 1 0 0]
[-2.  -1.6 -1.2 -0.8 -0.4  0.   0.4  0.8  1.2  1.6  2. ]
(array([1], dtype=int64), array([2], dtype=int64), array([3], dtype=int64), array([4], dtype=int64))
(array([1234], dtype=int64), array([0], dtype=int64))


In [16]:
arr = np.ones([2, 3])
print(arr)

np.reshape(arr, [-1, 1])

[[1. 1. 1.]
 [1. 1. 1.]]


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [17]:
# State 를 받아서 각 요소의 값들이 어느 부분(index)의 속하였고, 그 State를 가리키는 일렬로 나열시의 index값을 반환
def map_discrete_state(state):

    pos, vel, ang, anv = state
    
    idx_pos = np.where(np.histogram(np.clip(pos, -2, 2), bins = n_bins_pos, range=(-2,2))[0]==1)[0][0]
    idx_vel = np.where(np.histogram(np.clip(vel, -2, 2), bins = n_bins_vel, range=(-2,2))[0]==1)[0][0]
    idx_ang = np.where(np.histogram(np.clip(ang, -0.4, 0.4), bins = n_bins_ang, range=(-0.4,0.4))[0]==1)[0][0]
    idx_anv = np.where(np.histogram(np.clip(anv, -2, 2), bins = n_bins_anv, range=(-2,2))[0]==1)[0][0]
    
    states = np.zeros([n_bins_pos, n_bins_vel, n_bins_ang, n_bins_anv])
    states[idx_pos, idx_vel, idx_ang, idx_anv] = 1
    
    states = states.reshape(-1, 1) # 일렬로 세움 -> 당연히 2차원 배열로 늘어서게 되있음.. 1열로만 만들기 위해선 n 행이 필요!
    s = np.where(states == 1)[0][0]
    
    return s


In [18]:
np.where(np.histogram(np.clip(pos, -2, 2), bins = n_bins_pos, range=(-2,2))[0]==1)[0][0]

5

## Train Using Q-Learning

***process***  

1. Set the variables and Q-table


2. Get sample of State Random


3. Update Q-Table While Episode done
    1. Mapping the state to the index of Q-table (샘플이 어떤(몇번째) 스테이트인지 구함)
    2. Exploitation VS Exploration
    3. Update Q-Table According to Action for the next step (Using TD Algorithm / Q-Function)


In [20]:
alpha = 0.3
gamma = 0.9

Q_table = np.random.uniform(0, 1, (n_states, n_actions))

for episode in range(801):
    
    done = False
    
    state = env.reset()
    
    count = 0
    while not done:
        count += 1
        
        s = map_discrete_state(state)
        
        epsilon = 0.1
        if np.random.uniform() < epsilon:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q_table[s, :])
        
        next_state, reward, done, _ = env.step(a)
        
        if done:
            reward = -100
            Q_table[s, a] = reward
        else:
            next_s = map_discrete_state(next_state)
            Q_table[s, a] = (1-alpha)*Q_table[s,a] + (alpha)*(reward + gamma*np.max(Q_table[next_s, :]))        
        
        state = next_state
    
    if episode % 100 == 0:
        print(episode, count)
env.close()

0 15
100 73
200 84
300 200
400 47
500 200
600 186
700 146
800 200


## Test The Result

In [23]:
state = env.reset()

done = False

while not done:
    env.render()
    
    s = map_discrete_state(state)
    a = np.argmax(Q_table[s, :])
    
    next_state, _, done, _ = env.step(a)
    
    state = next_state
    
env.close()